<a href="https://colab.research.google.com/github/brendanpshea/programming_problem_solving/blob/main/Java_09_FilesExceptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction - Welcome to the Bank of Evil

Imagine you're writing software for the Bank of Evil, where cartoon villains keep their stolen money. Gru needs to withdraw funds for his moon heist. Vector wants a loan for piranha guns. Your banking system needs to handle all of this... but what happens when things go wrong?

Right now, when your programs hit an error, they crash with a scary error message. That's not acceptable for real software. This chapter teaches you how to write programs that expect problems and handle them gracefully. This is called **defensive programming**.

## Your First Taste of Trouble

Let's see what happens when a program encounters a problem:

```java
public class EvilBankDemo {
    public static void main(String[] args) {
        String[] villains = {"Gru", "Vector", "Scarlet"};
        System.out.println("Villain #4: " + villains[3]);
    }
}
```

This crashes! Java shows an `ArrayIndexOutOfBoundsException` because we tried to access position 3, but the array only has positions 0, 1, and 2.

When Java encounters a problem it can't handle, it creates an **exception** - Java's way of saying "Something went wrong!" Right now, exceptions crash your program. By the end of this chapter, you'll know how to catch and handle them properly.

## What You'll Build

We'll create a simple Bank of Evil system that:
- Stores villain account balances in files
- Handles deposits and withdrawals
- Deals with problems like insufficient funds without crashing
- Uses inheritance to create different account types (building on last chapter's OOP concepts)

Here's a preview of where we're heading:

```java
try {
    account.withdraw(1000000);
    System.out.println("Withdrawal successful!");
} catch (InsufficientEvilFundsException e) {
    System.out.println("Sorry, not enough funds for world domination.");
}
```

This code tries to withdraw money. If there aren't enough funds, it catches the problem and prints a helpful message instead of crashing. The program keeps running, ready for the next villain.

Let's learn how to make your programs villain-proof!

# Saving Villain Data - Your First Files

The Bank of Evil needs to remember account balances even after the program ends. When Gru deposits money today, it should still be there tomorrow. That's where files come in. Java programs can save data to files and read it back later.

## Writing to a File

To write to files in Java, we use a class called `PrintWriter`. This class works almost exactly like `System.out.println()`, but instead of printing to the screen, it prints to a file.

First, you need to import the necessary classes at the top of your program:

```java
import java.io.PrintWriter;
import java.io.FileNotFoundException;
```

The first import gives us `PrintWriter`. The second import is for something called `FileNotFoundException` - an exception that Java requires us to handle when working with files. For now, we'll add `throws FileNotFoundException` after our method name to tell Java "we know this might go wrong":

```java
public static void main(String[] args) throws FileNotFoundException {
    PrintWriter writer = new PrintWriter("villains.txt");
    writer.println("Gru");
    writer.close();
}
```

This creates a file named "villains.txt" and writes "Gru" to it. The `close()` method is crucial - it saves your data and releases the file so other programs can use it. Think of it like saving a document before closing it.

## Adding More Data

Once you have a `PrintWriter`, you can write as many lines as you want:

```java
PrintWriter writer = new PrintWriter("villains.txt");
writer.println("Gru");
writer.println("Vector");
writer.println("Scarlet");
writer.close();
```

Each `println()` adds a new line to the file. When you run this program, it creates a text file with three villain names, each on its own line.

## Reading from a File

Reading files uses `Scanner`, the same class you've used for keyboard input. But instead of reading from `System.in`, we tell it to read from a file.

First, import what we need:

```java
import java.util.Scanner;
import java.io.File;
import java.io.FileNotFoundException;
```

The `File` class represents a file on your computer. We create a `File` object, then pass it to `Scanner`:

```java
public static void main(String[] args) throws FileNotFoundException {
    File inputFile = new File("villains.txt");
    Scanner reader = new Scanner(inputFile);
}
```

Now `reader` reads from the file instead of the keyboard. To read one line:

```java
String villain = reader.nextLine();
System.out.println("Found: " + villain);
reader.close();
```

## Reading All Lines

Files can have many lines, and we often don't know how many. The `hasNextLine()` method checks if there's more to read:

```java
File inputFile = new File("villains.txt");
Scanner reader = new Scanner(inputFile);

while (reader.hasNextLine()) {
    String villain = reader.nextLine();
    System.out.println("Found villain: " + villain);
}
reader.close();
```

The loop continues until it reaches the end of the file. This pattern - checking `hasNextLine()` before calling `nextLine()` - prevents errors.

## CSV Format for Multiple Values

The Bank of Evil needs to store more than just names. We need account numbers and balances too. CSV (Comma-Separated Values) is perfect for this. It's just a text file where each line contains values separated by commas.

Here's what CSV data looks like:
```
1001,Gru,50000.00
1002,Vector,75000.00
```

Each line has three pieces of information: account number, name, and balance. To write CSV data:

```java
PrintWriter writer = new PrintWriter("accounts.csv");
writer.println("1001,Gru,50000.00");
writer.println("1002,Vector,75000.00");
writer.close();
```

## Reading CSV Data

To read CSV data, we read a line and then split it into parts. Java's `split()` method breaks a string at every comma:

```java
String line = "1001,Gru,50000.00";
String[] parts = line.split(",");
// Now: parts[0] is "1001"
//      parts[1] is "Gru"  
//      parts[2] is "50000.00"
```

The `split(",")` method returns an array of strings. We can then access each piece by its index.

## Complete Example: Villain Account Manager

Let's put it all together. This program saves villain accounts to a file and reads them back:

In [1]:
%%writefile VillainAccounts.java
import java.io.*;
import java.util.Scanner;

public class VillainAccounts {
    public static void main(String[] args) throws FileNotFoundException {
        // Part 1: Save villain accounts to a file
        PrintWriter writer = new PrintWriter("accounts.csv");

        // Write a header line (this helps us remember what each column means)
        writer.println("AccountNumber,Name,Balance");

        // Write three villain accounts
        writer.println("1001,Gru,50000.00");
        writer.println("1002,Vector,75000.00");
        writer.println("1003,Scarlet,100000.00");

        writer.close();  // Don't forget to close!
        System.out.println("Accounts saved to file.");

        // Part 2: Read the accounts back
        System.out.println("\nReading accounts from file:");

        File inputFile = new File("accounts.csv");
        Scanner reader = new Scanner(inputFile);

        // Skip the header line - we don't need to process it
        reader.nextLine();

        // Read each account
        while (reader.hasNextLine()) {
            String line = reader.nextLine();
            String[] parts = line.split(",");

            // Extract the three pieces of information
            String accountNum = parts[0];
            String name = parts[1];
            String balance = parts[2];

            // Display in a nice format
            System.out.println("Account " + accountNum + ": " +
                             name + " has $" + balance);
        }

        reader.close();
    }
}


Writing VillainAccounts.java


In [2]:
!javac VillainAccounts.java
!java VillainAccounts

Accounts saved to file.

Reading accounts from file:
Account 1001: Gru has $50000.00
Account 1002: Vector has $75000.00
Account 1003: Scarlet has $100000.00



This program demonstrates the complete file cycle: creating data, saving it to a file, and reading it back. The CSV format lets us store structured information that our Bank of Evil system can use.

Notice that we're still using `throws FileNotFoundException` on our main method. This means if something goes wrong (like the file doesn't exist), the program will crash. In the next section, we'll learn what these exceptions really are and how to handle them gracefully.

Try this: Run the program once to create the file. Then comment out the writing part and run it again - it still reads the data! The file persists between program runs. Now try changing "accounts.csv" in the reading part to "missing.csv" - what happens?

# When Things Go Wrong - Meet the Exception

You've been seeing `throws FileNotFoundException` in every file example. It's time to understand what this means. When Java encounters a problem it can't solve, it creates an **exception** - a special object that contains information about what went wrong.

## Your Programs Already Throw Exceptions

You've probably seen exceptions before, even if you didn't know what they were called. Let's create one on purpose:

```java
int[] villainScores = {10, 20, 30};
System.out.println(villainScores[5]);  // Crash!
```

This crashes with an `ArrayIndexOutOfBoundsException`. The array has only 3 elements (at positions 0, 1, and 2), but we tried to access position 5.

Here's another common exception:

```java
String villainName = null;
int length = villainName.length();  // Crash!
```

This creates a `NullPointerException`. We're trying to call a method on something that doesn't exist (null). It's like trying to check the balance of an account that was never created.

## Understanding Error Messages

When an exception occurs, Java prints a helpful error message. Let's learn to read it:

```java
public class BankCrash {
    public static void main(String[] args) {
        String[] villains = {"Gru", "Vector"};
        System.out.println(villains[10]);
    }
}
```

This produces an error message like:
```
Exception in thread "main" java.lang.ArrayIndexOutOfBoundsException: 10
    at BankCrash.main(BankCrash.java:4)
```

This tells us three important things:
1. **What went wrong**: ArrayIndexOutOfBoundsException at index 10
2. **Where it happened**: BankCrash.java, line 4
3. **Which method**: in the main method

## File Exceptions

Files introduce many opportunities for exceptions. The most common is `FileNotFoundException`:

```java
File inputFile = new File("missing.txt");
Scanner reader = new Scanner(inputFile);  // Crash if file doesn't exist!
```

If "missing.txt" doesn't exist, Java throws a `FileNotFoundException`. This is why we've been adding `throws FileNotFoundException` to our methods - it tells Java "we know this might happen."

## The Exception Family Tree

Just like classes can inherit from other classes, exceptions form a family tree. All exceptions inherit from a class called `Exception`. Here are the most common ones you'll encounter:

- **NullPointerException**: Using something that's null
- **ArrayIndexOutOfBoundsException**: Going outside array bounds  
- **FileNotFoundException**: File doesn't exist
- **ArithmeticException**: Math problems like dividing by zero

Let's cause an `ArithmeticException`:

```java
int villainCount = 0;
int fundsPerVillain = 100000 / villainCount;  // Crash! Can't divide by zero
```

## Exceptions Stop Your Program

When an exception occurs, your program stops immediately at that line. Any code after it doesn't run:

```java
System.out.println("Starting villain transfer...");
String[] accounts = {"Gru", "Vector"};
System.out.println(accounts[10]);  // Exception happens here
System.out.println("Transfer complete!");  // This NEVER prints
```

The last line never executes because the program crashes on line 3. This is a problem for the Bank of Evil - we can't have the entire system crash just because one villain makes a mistake!

## Methods Pass Exceptions Up

When a method encounters an exception it doesn't handle, it passes the exception to whoever called it. This continues until someone handles it or the program crashes:

```java
public static void checkBalance(int accountIndex) {
    String[] accounts = {"Gru", "Vector"};
    System.out.println(accounts[accountIndex]);  // Exception might happen here
}

public static void main(String[] args) {
    checkBalance(10);  // Exception gets passed here, then crash
}
```

The exception starts in `checkBalance` but gets passed to `main`. Since `main` doesn't handle it either, the program crashes.

## Complete Example: The Problematic Bank

Here's a program that demonstrates several exceptions. Run it and try different inputs to see various crashes:


In [3]:
%%writefile ProblematicBank.java
import java.util.Scanner;
import java.io.*;

public class ProblematicBank {
    public static void main(String[] args) throws FileNotFoundException {
        Scanner keyboard = new Scanner(System.in);

        // This array represents our account database
        String[] villains = {"Gru", "Vector", "Scarlet"};
        double[] balances = {50000, 75000, 100000};

        System.out.println("=== BANK OF EVIL - PROBLEMATIC VERSION ===");
        System.out.println("We have " + villains.length + " accounts.");

        // Problem 1: What if user enters a number too big?
        System.out.print("\nEnter account number (0-2): ");
        int accountNum = keyboard.nextInt();

        // This line might crash with ArrayIndexOutOfBoundsException
        System.out.println("Account " + accountNum + ": " + villains[accountNum]);
        System.out.println("Balance: $" + balances[accountNum]);

        // Problem 2: What if user enters zero?
        System.out.print("\nHow many villains to split the funds between? ");
        int villainCount = keyboard.nextInt();

        // This might crash with ArithmeticException if villainCount is 0
        double amountEach = 100000.0 / villainCount;
        System.out.println("Each villain gets: $" + amountEach);

        // Problem 3: What if file doesn't exist?
        System.out.print("\nEnter filename to load: ");
        keyboard.nextLine();  // Clear the buffer
        String filename = keyboard.nextLine();

        // This might crash with FileNotFoundException
        File file = new File(filename);
        Scanner fileReader = new Scanner(file);
        System.out.println("First line: " + fileReader.nextLine());
        fileReader.close();

        System.out.println("\nBanking complete!");
    }
}


Writing ProblematicBank.java


In [4]:
!javac ProblematicBank.java
!java ProblematicBank

=== BANK OF EVIL - PROBLEMATIC VERSION ===
We have 3 accounts.

Enter account number (0-2): 3
Exception in thread "main" java.lang.ArrayIndexOutOfBoundsException: Index 3 out of bounds for length 3
	at ProblematicBank.main(ProblematicBank.java:20)



Try these inputs to see different exceptions:
- Enter 10 for account number (ArrayIndexOutOfBoundsException)
- Enter 0 for villain count (ArithmeticException)  
- Enter "fake.txt" for filename (FileNotFoundException)
- Enter -1 for account number (still ArrayIndexOutOfBoundsException!)

Each problem crashes the entire program. The "Banking complete!" message never appears when an exception occurs. This is unacceptable for a real banking system!

## What's Next?

Right now, exceptions are your enemy - they crash your program. In the next section, you'll learn to catch these exceptions and handle them gracefully. Instead of crashing when a villain enters an invalid account number, your program will display a helpful error message and keep running. The Bank of Evil will become crash-proof!

# Try and Catch - Catching Problems

Right now, exceptions crash your program. But Java gives you a way to catch exceptions and handle them gracefully. This is done with a `try-catch` block. Think of it as putting a safety net under dangerous code.

## Your First Try-Catch

The basic structure looks like this:

```java
try {
    // Dangerous code goes here
} catch (Exception e) {
    // What to do if something goes wrong
}
```

The `try` block contains code that might throw an exception. If an exception occurs, Java immediately jumps to the `catch` block. Let's fix our array problem from before:

```java
String[] villains = {"Gru", "Vector", "Scarlet"};

try {
    System.out.println(villains[10]);
} catch (Exception e) {
    System.out.println("That villain doesn't exist!");
}

System.out.println("Program continues running!");
```

Now instead of crashing, the program prints "That villain doesn't exist!" and continues. The last line executes normally.

## How Try-Catch Works

When Java executes a try-catch:
1. It starts running the code in the `try` block
2. If no exception occurs, it skips the `catch` block entirely
3. If an exception occurs, it immediately jumps to the `catch` block
4. After the catch block (or try block if no exception), the program continues

Here's an example that shows this flow:

```java
System.out.println("1. Before try");

try {
    System.out.println("2. Inside try - before problem");
    int result = 10 / 0;  // Exception happens here!
    System.out.println("3. This NEVER prints");
} catch (Exception e) {
    System.out.println("4. Caught a problem!");
}

System.out.println("5. After try-catch");
```

This prints: 1, 2, 4, 5. Notice that step 3 never executes - when the exception occurs, Java immediately jumps to the catch block.

## The Exception Variable

The catch block receives an exception object (we usually call it `e`). This object contains information about what went wrong:

```java
try {
    int[] scores = {10, 20};
    System.out.println(scores[5]);
} catch (Exception e) {
    System.out.println("Error: " + e.getMessage());
}
```

The `getMessage()` method returns a description of the problem. For an array index error, it might return "Index 5 out of bounds for length 2".

## Catching File Exceptions

Remember how we've been using `throws FileNotFoundException`? Now we can handle it properly:

```java
try {
    File file = new File("missing.txt");
    Scanner reader = new Scanner(file);
    System.out.println(reader.nextLine());
    reader.close();
} catch (Exception e) {
    System.out.println("Couldn't read file: " + e.getMessage());
}
```

If the file doesn't exist, instead of crashing, the program displays a helpful message.

## Catching Specific Exceptions

You can catch specific types of exceptions. This lets you handle different problems differently:

```java
try {
    // Some dangerous code
} catch (FileNotFoundException e) {
    System.out.println("File not found!");
} catch (ArrayIndexOutOfBoundsException e) {
    System.out.println("Invalid account number!");
}
```

Java checks each catch block in order. If the exception matches, that catch block runs. You can also have a general catch-all at the end:

```java
try {
    // Dangerous code
} catch (FileNotFoundException e) {
    System.out.println("File not found!");
} catch (Exception e) {
    System.out.println("Something else went wrong: " + e.getMessage());
}
```

## When to Use Try-Catch

Use try-catch when:
- Working with files (they might not exist)
- Converting strings to numbers (user might type "abc" instead of "123")
- Accessing arrays with user-provided indices
- Any time your program interacts with the outside world

Don't wrap your entire program in one giant try-catch. Instead, catch exceptions close to where they might occur and handle them appropriately.

## Complete Example: The Improved Bank

Let's fix our problematic bank from the last section:


In [5]:
%%writefile ImprovedBank.java
import java.util.Scanner;
import java.io.*;

public class ImprovedBank {
    public static void main(String[] args) {
        Scanner keyboard = new Scanner(System.in);

        // Our account database
        String[] villains = {"Gru", "Vector", "Scarlet"};
        double[] balances = {50000, 75000, 100000};

        System.out.println("=== BANK OF EVIL - IMPROVED VERSION ===");
        System.out.println("We have " + villains.length + " accounts.");

        // Problem 1: Handling invalid account numbers
        System.out.print("\nEnter account number (0-2): ");
        int accountNum = keyboard.nextInt();

        try {
            System.out.println("Account " + accountNum + ": " + villains[accountNum]);
            System.out.println("Balance: $" + balances[accountNum]);
        } catch (ArrayIndexOutOfBoundsException e) {
            System.out.println("Error: Account " + accountNum + " doesn't exist!");
            System.out.println("Valid accounts are 0 to " + (villains.length - 1));
        }

        // Problem 2: Handling division by zero
        System.out.print("\nHow many villains to split the funds between? ");
        int villainCount = keyboard.nextInt();

        try {
            double amountEach = 100000.0 / villainCount;
            System.out.println("Each villain gets: $" + amountEach);
        } catch (ArithmeticException e) {
            System.out.println("Error: Can't split between 0 villains!");
        }

        // Problem 3: Handling missing files
        System.out.print("\nEnter filename to load: ");
        keyboard.nextLine();  // Clear the buffer
        String filename = keyboard.nextLine();

        try {
            File file = new File(filename);
            Scanner fileReader = new Scanner(file);
            System.out.println("First line: " + fileReader.nextLine());
            fileReader.close();
            System.out.println("File loaded successfully!");
        } catch (FileNotFoundException e) {
            System.out.println("Error: File '" + filename + "' not found!");
            System.out.println("Please check the filename and try again.");
        }

        // This line ALWAYS executes now, even after errors!
        System.out.println("\nBanking session complete!");
    }
}


Writing ImprovedBank.java


In [6]:
!javac ImprovedBank.java
!java ImprovedBank

=== BANK OF EVIL - IMPROVED VERSION ===
We have 3 accounts.

Enter account number (0-2): 3
Error: Account 3 doesn't exist!
Valid accounts are 0 to 2

How many villains to split the funds between? 0
Each villain gets: $Infinity

Enter filename to load: sjds
Error: File 'sjds' not found!
Please check the filename and try again.

Banking session complete!



Now try the same problematic inputs:
- Enter 10 for account number - you get a helpful error message
- Enter 0 for villain count - you get a different error message
- Enter "fake.txt" for filename - you get a file error message

Most importantly, the program never crashes! The "Banking session complete!" message always appears. Each problem is handled gracefully with a helpful message to the user.

## The Power of Not Crashing

The Bank of Evil can now handle villain mistakes without bringing down the entire system. When Gru enters an invalid account number, he gets a helpful message and can try again. When Vector tries to divide funds between zero minions, the system explains the problem instead of crashing.

In the next section, we'll learn to create our own custom exceptions for Bank of Evil-specific problems, like "InsufficientEvilFundsException" when a villain tries to withdraw too much money!

# Creating Your Own Exceptions

Java's built-in exceptions are useful, but the Bank of Evil has special problems that need special exceptions. When a villain tries to withdraw more money than they have, we don't want a generic "Exception" - we want an "InsufficientEvilFundsException"!

## Your First Custom Exception

Creating a custom exception is surprisingly simple. It's just a class that extends `Exception`:

```java
public class InsufficientEvilFundsException extends Exception {
    // That's it! This is a complete custom exception
}
```

The `extends Exception` part makes your class an exception. Java now recognizes it as something that can be thrown and caught.

## Adding a Message

Exceptions become more helpful when they include messages. Add a constructor that accepts a message:

```java
public class InsufficientEvilFundsException extends Exception {
    public InsufficientEvilFundsException(String message) {
        super(message);  // Pass the message to Exception's constructor
    }
}
```

The `super(message)` calls the parent class constructor, storing the message so it can be retrieved with `getMessage()`.

## Throwing Your Exception

To create and throw an exception, use the `throw` keyword (not `throws` - that's different!):

```java
if (withdrawAmount > balance) {
    throw new InsufficientEvilFundsException("Not enough funds!");
}
```

Notice we create a new exception object with `new`, then `throw` it. When Java encounters a `throw` statement, it immediately stops and looks for a catch block.

## The Complete Exception Class

Let's create a more complete exception for the Bank of Evil:

```java
public class InsufficientEvilFundsException extends Exception {
    private double requested;
    private double available;
    
    public InsufficientEvilFundsException(double requested, double available) {
        super("Insufficient funds: Requested $" + requested +
              " but only $" + available + " available");
        this.requested = requested;
        this.available = available;
    }
    
    public double getShortfall() {
        return requested - available;
    }
}
```

This exception stores additional information about what went wrong. The `getShortfall()` method tells us how much money the villain is short.

## Using Your Custom Exception

Here's how to use the exception in a withdraw method:

```java
public static void withdraw(double balance, double amount)
        throws InsufficientEvilFundsException {
    if (amount > balance) {
        throw new InsufficientEvilFundsException(amount, balance);
    }
    System.out.println("Withdrawal approved!");
}
```

Notice the `throws InsufficientEvilFundsException` in the method header. This tells Java (and other programmers) that this method might throw this exception.

## Catching Custom Exceptions

Custom exceptions are caught just like built-in ones:

```java
try {
    withdraw(1000.00, 5000.00);  // Try to withdraw $5000 from $1000 balance
} catch (InsufficientEvilFundsException e) {
    System.out.println(e.getMessage());
    System.out.println("You need $" + e.getShortfall() + " more!");
}
```

## Creating Multiple Custom Exceptions

The Bank of Evil needs several custom exceptions. Here's another one:

```java
public class VillainNotFoundException extends Exception {
    public VillainNotFoundException(String villainId) {
        super("Villain with ID '" + villainId + "' not found in system");
    }
}
```

Now we can throw different exceptions for different problems:

```java
public static double getBalance(String villainId)
        throws VillainNotFoundException {
    if (!villainExists(villainId)) {
        throw new VillainNotFoundException(villainId);
    }
    // Return balance...
}
```

## Why Custom Exceptions Matter

Custom exceptions make your code more professional and easier to debug. Instead of a generic "Exception", other programmers (and future you) know exactly what went wrong. "InsufficientEvilFundsException" is much clearer than "Exception"!

Custom exceptions also let you add problem-specific information. Our InsufficientEvilFundsException can tell us exactly how much money is missing. A generic exception can't do that.

In the next section, we'll learn defensive programming - how to check for problems before they happen, so we can avoid throwing exceptions in the first place!

# Defensive Programming - Check First, Crash Never

It's better to prevent problems than to catch them after they happen. Defensive programming means checking for potential problems before they occur. Think of it like a villain checking for security cameras before robbing a bank - preparation prevents problems!

## Validate Input Immediately

Never trust input from users. They might type anything! Always check if input makes sense before using it:

```java
System.out.print("Enter withdrawal amount: ");
double amount = keyboard.nextDouble();

if (amount <= 0) {
    System.out.println("Amount must be positive!");
    return;  // Exit early
}
```

This is better than letting a negative amount cause problems later in your program.

## Check Before You Use

Before using an array index, check if it's valid:

```java
System.out.print("Enter account number: ");
int index = keyboard.nextInt();

if (index < 0 || index >= accounts.length) {
    System.out.println("Invalid account number!");
} else {
    System.out.println("Balance: " + accounts[index]);
}
```

By checking first, we avoid an `ArrayIndexOutOfBoundsException`. The program handles the problem gracefully instead of crashing.

## Guard Against Null

Null causes many crashes. Always check if something might be null before using it:

```java
String villainName = getVillainName();  // Might return null

if (villainName != null) {
    System.out.println("Welcome, " + villainName);
} else {
    System.out.println("Villain not found");
}
```

Without the null check, calling any method on `villainName` would crash with a `NullPointerException`.

## Initialize Variables Properly

Don't leave variables uninitialized. Give them safe default values:

```java
// Bad - might be used before initialization
String villainPlan;

// Good - has a safe default
String villainPlan = "No plan yet";
```

Even better, initialize arrays and objects immediately:

```java
// Instead of this:
double[] balances;
// Later... balances[0] = 100;  // Crash! Array not created

// Do this:
double[] balances = new double[10];  // Safe to use immediately
```

## Validate Method Parameters

Check parameters at the start of your methods:

```java
public static void withdraw(double balance, double amount) {
    // Check parameters first
    if (amount <= 0) {
        System.out.println("Withdrawal amount must be positive");
        return;
    }
    
    if (amount > balance) {
        System.out.println("Insufficient funds");
        return;
    }
    
    // Now safe to proceed
    System.out.println("Withdrawal successful");
}
```

By checking early, you catch problems before they cause damage.

## Use Helper Methods for Validation

Create methods specifically for validation. This keeps your code organized:

```java
public static boolean isValidAccountNumber(int number) {
    return number >= 0 && number < accounts.length;
}

public static boolean isValidAmount(double amount) {
    return amount > 0 && amount <= 1000000;  // Max withdrawal is $1M
}
```

Now you can use these anywhere:

```java
if (!isValidAmount(withdrawAmount)) {
    System.out.println("Amount must be between $0 and $1,000,000");
    return;
}
```

## Range Checking

When working with numbers, always consider the valid range:

```java
System.out.print("Enter villain threat level (1-10): ");
int threatLevel = keyboard.nextInt();

// Defensive check
if (threatLevel < 1) {
    threatLevel = 1;  // Minimum threat
} else if (threatLevel > 10) {
    threatLevel = 10;  // Maximum threat
}
```

This ensures `threatLevel` is always valid, no matter what the user enters.

## File Defensive Strategies

Before working with files, check if they exist and are readable:

```java
File dataFile = new File("villains.txt");

if (!dataFile.exists()) {
    System.out.println("Creating new villain database...");
    // Create the file or use defaults
} else if (!dataFile.canRead()) {
    System.out.println("Cannot read villain database!");
    return;
} else {
    // Safe to read the file
}
```

## The Fail-Fast Principle

If something is wrong, detect it and report it immediately. Don't let bad data spread through your program:

```java
public static void processVillainData(String data) {
    // Check immediately
    if (data == null || data.isEmpty()) {
        System.out.println("ERROR: No data provided");
        return;  // Exit immediately
    }
    
    // Check format
    String[] parts = data.split(",");
    if (parts.length != 3) {
        System.out.println("ERROR: Data must have exactly 3 parts");
        return;  // Exit immediately
    }
    
    // Now safe to process
    // ... rest of method
}
```

## Practice: Refactor This Dangerous Code

Here's a program full of potential crashes. Your task: add defensive programming to make it safe. Look for places where the code might crash and add appropriate checks:


In [ ]:
%%writefile DangerousBank.java
import java.util.Scanner;
import java.io.*;

public class DangerousBank {
    static String[] villains = {"Gru", "Vector", "Scarlet"};
    static double[] balances = {1000, 2000, 3000};

    public static void displayAccount(int index) {
        // DANGER: No bounds checking!
        System.out.println(villains[index] + ": $" + balances[index]);
    }

    public static void transfer(int from, int to, double amount) {
        // DANGER: No validation at all!
        balances[from] = balances[from] - amount;
        balances[to] = balances[to] + amount;
        System.out.println("Transfer complete!");
    }

    public static void saveToFile(String filename) throws FileNotFoundException {
        // DANGER: What if filename is null?
        PrintWriter writer = new PrintWriter(filename);

        // DANGER: Assumes arrays are same length
        for (int i = 0; i < villains.length; i++) {
            writer.println(villains[i] + "," + balances[i]);
        }
        writer.close();
    }

    public static double calculateAverage(double[] values) {
        // DANGER: What if array is null or empty?
        double sum = 0;
        for (double val : values) {
            sum += val;
        }
        return sum / values.length;  // DANGER: Might divide by zero!
    }

    public static void main(String[] args) throws FileNotFoundException {
        Scanner keyboard = new Scanner(System.in);

        System.out.print("Enter account to view: ");
        int account = keyboard.nextInt();
        displayAccount(account);  // DANGER: Might be invalid!

        System.out.print("Transfer from account: ");
        int from = keyboard.nextInt();
        System.out.print("Transfer to account: ");
        int to = keyboard.nextInt();
        System.out.print("Amount: ");
        double amount = keyboard.nextDouble();

        transfer(from, to, amount);  // DANGER: No validation!

        System.out.print("Save to file: ");
        keyboard.nextLine();  // Clear buffer
        String filename = keyboard.nextLine();
        saveToFile(filename);  // DANGER: Might be empty string!

        double avg = calculateAverage(balances);
        System.out.println("Average balance: $" + avg);
    }
}

In [ ]:
!javac DangerousBank.java
!java DangerousBank


This code has at least 10 potential crashes. Can you find and fix them all? Add defensive checks to prevent:
- Array index out of bounds
- Null pointer exceptions
- Division by zero
- Invalid amounts (negative transfers)
- File problems
- Logic errors (transferring more than available)

Remember: It's better to check and give a helpful message than to let the program crash!

# Using Inheritance with Exceptions

Remember how you learned about inheritance in the last chapter? Exceptions use inheritance too! In fact, every exception you've seen is part of a family tree. Let's see how this connects to the Bank of Evil's account system.

## Exceptions Are Classes Too

When you created `InsufficientEvilFundsException extends Exception`, you were using inheritance! Your custom exception is a child class of `Exception`. This means it inherits all the abilities of Exception, like storing error messages.

```java
// Exception is the parent
// InsufficientEvilFundsException is the child
public class InsufficientEvilFundsException extends Exception {
    // Inherits getMessage() and other methods from Exception
}
```

## Different Accounts, Different Rules

At the Bank of Evil, different account types handle problems differently. Let's create a simple account hierarchy:

```java
public class Account {
    protected double balance;
    protected String owner;
    
    public void withdraw(double amount) throws InsufficientEvilFundsException {
        if (amount > balance) {
            throw new InsufficientEvilFundsException("Not enough funds");
        }
        balance -= amount;
    }
}
```

Now let's create two child classes with different behaviors:

```java
public class VillainAccount extends Account {
    // Villains can overdraft up to $1000
    @Override
    public void withdraw(double amount) throws InsufficientEvilFundsException {
        if (amount > balance + 1000) {  // Allow $1000 overdraft
            throw new InsufficientEvilFundsException("Exceeds overdraft limit");
        }
        balance -= amount;
    }
}

public class MinionAccount extends Account {
    // Minions cannot overdraft at all - uses parent's strict rules
    // No need to override - parent's method is perfect
}
```

## Same Method, Different Behaviors

The beauty of inheritance is that we can treat all accounts the same way, but each behaves according to its type:

```java
Account gruAccount = new VillainAccount();
Account bobAccount = new MinionAccount();

try {
    gruAccount.withdraw(5000);  // Might work with overdraft
    bobAccount.withdraw(5000);  // Will fail if balance is less
} catch (InsufficientEvilFundsException e) {
    System.out.println(e.getMessage());
}
```

Both accounts have a `withdraw` method that might throw the same exception, but the rules for when they throw it are different!

## Exception Inheritance Hierarchy

Just like your account classes, Java's exceptions form a hierarchy:

```
Exception (parent of all exceptions)
    ├── IOException (file problems)
    │   └── FileNotFoundException
    ├── RuntimeException (runtime problems)
    │   ├── NullPointerException
    │   └── ArrayIndexOutOfBoundsException
    └── InsufficientEvilFundsException (your custom exception)
```

This hierarchy is useful because you can catch parent types to handle multiple child types:

```java
try {
    // Some code that might throw various exceptions
} catch (IOException e) {
    // This catches FileNotFoundException and any other IOException
    System.out.println("File problem: " + e.getMessage());
} catch (Exception e) {
    // This catches EVERYTHING else
    System.out.println("Some other problem: " + e.getMessage());
}
```

## Polymorphism with Error Handling

Here's where it gets powerful. You can have an array of different account types, and each handles errors its own way:

```java
Account[] bankAccounts = {
    new VillainAccount(),   // Gru
    new MinionAccount(),     // Bob
    new VillainAccount(),   // Vector
    new MinionAccount()      // Kevin
};

// Process all accounts the same way
for (Account acc : bankAccounts) {
    try {
        acc.withdraw(2000);
        System.out.println("Withdrawal successful");
    } catch (InsufficientEvilFundsException e) {
        System.out.println("Failed: " + e.getMessage());
    }
}
```

Each account follows its own rules, but we handle them all with the same code. That's the power of combining inheritance with exception handling!

## The Big Picture

Why does this matter? In real programs, you'll often have:
- Different types of objects (like accounts) that follow different rules
- The same operations (like withdraw) that might fail in different ways
- The need to handle errors consistently across all types

Inheritance lets you:
1. Write error handling code once
2. Let each subclass define its own rules for when errors occur
3. Treat all objects uniformly while respecting their differences

Think of it like the Bank of Evil's security system. There's one alarm system (the exception), but different vaults (account types) trigger it under different conditions. The response to the alarm (catch block) is the same, but when it triggers depends on the specific vault's rules.

Remember: The goal isn't to memorize every detail about exception inheritance. It's to understand that exceptions are just classes, and everything you learned about inheritance applies to them too!

# The Finally Block - Cleaning Up

Sometimes you need code to run no matter what happens - whether everything works perfectly or an exception occurs. That's what the `finally` block is for. Think of it like the Bank of Evil's vault door - it must close whether the heist succeeds or fails!

## The Finally Block Structure

The `finally` block comes after your catch blocks:

```java
try {
    // Dangerous code
} catch (Exception e) {
    // Handle problems
} finally {
    // This ALWAYS runs
}
```

The code in `finally` runs no matter what:
- If the try block succeeds
- If an exception is caught
- Even if an exception isn't caught!

## A Simple Example

Let's see `finally` in action:

```java
System.out.println("Opening vault...");

try {
    System.out.println("Grabbing money...");
    int[] vault = {1000, 2000};
    System.out.println("Found: $" + vault[5]);  // This will fail!
} catch (Exception e) {
    System.out.println("Heist failed!");
} finally {
    System.out.println("Closing vault door!");
}

System.out.println("Running away...");
```

This prints:
```
Opening vault...
Grabbing money...
Heist failed!
Closing vault door!
Running away...
```

The vault door closes even though the heist failed!

## Why Finally Matters: File Cleanup

The most common use of `finally` is closing files. When you open a file, you should always close it - even if something goes wrong:

```java
PrintWriter writer = null;

try {
    writer = new PrintWriter("villain_log.txt");
    writer.println("Starting evil plan...");
    // Something might go wrong here
    writer.println("Plan complete!");
} catch (Exception e) {
    System.out.println("Plan failed: " + e.getMessage());
} finally {
    if (writer != null) {
        writer.close();  // Always close the file!
    }
}
```

Without the `finally` block, if an exception occurs, the file might never close. This can cause problems:
- The data might not be saved
- Other programs can't access the file
- You might run out of file handles

## Finally Without Catch

You don't always need a catch block. Sometimes you just want cleanup:

```java
Scanner fileReader = null;

try {
    fileReader = new Scanner(new File("accounts.csv"));
    // Read and process file
} finally {
    if (fileReader != null) {
        fileReader.close();  // Always close, even if we don't catch exceptions
    }
}
```

This ensures the file closes even though we're not catching exceptions. The exception still bubbles up to be handled elsewhere, but the cleanup happens first.

## What Happens When

The order matters. Here's exactly when each part runs:

```java
System.out.println("1. Before try");

try {
    System.out.println("2. Inside try");
    throw new Exception("Boom!");
    // System.out.println("3. Never prints");
} catch (Exception e) {
    System.out.println("4. Inside catch");
} finally {
    System.out.println("5. Inside finally");
}

System.out.println("6. After everything");
```

Output: 1, 2, 4, 5, 6

The finally block (5) runs after the catch (4) but before continuing with the rest of the program (6).

## Common Finally Uses

Here are the most common things to put in finally blocks:

1. **Closing files** - Always close files you've opened
2. **Closing database connections** - Return connections to the pool
3. **Releasing locks** - Unlock resources for other programs
4. **Logging** - Record that an operation finished (successfully or not)

```java
// Example: Logging transaction attempts
try {
    performTransaction();
    logToFile("Transaction successful");
} catch (Exception e) {
    logToFile("Transaction failed: " + e.getMessage());
} finally {
    logToFile("Transaction attempt completed at " + new Date());
}
```

## A Complete Example: The Transaction Logger

Here's a practical example that logs all Bank of Evil transactions, successful or not:

In [7]:
%%writefile TransactionLogger.java
import java.io.*;
import java.util.Date;

public class TransactionLogger {

    public static void logTransaction(String villain, double amount) {
        PrintWriter logger = null;

        try {
            // Open log file in append mode
            FileWriter fw = new FileWriter("transactions.log", true);
            logger = new PrintWriter(fw);

            // Log the attempt
            logger.println("=== TRANSACTION START ===");
            logger.println("Time: " + new Date());
            logger.println("Villain: " + villain);
            logger.println("Amount: $" + amount);

            // Simulate the transaction (might fail)
            if (amount > 10000) {
                throw new Exception("Amount too large for standard transfer");
            }

            // If we get here, transaction succeeded
            logger.println("Status: SUCCESS");
            System.out.println("Transaction successful!");

        } catch (Exception e) {
            // Log the failure
            if (logger != null) {
                logger.println("Status: FAILED - " + e.getMessage());
            }
            System.out.println("Transaction failed: " + e.getMessage());

        } finally {
            // Always close the log file
            if (logger != null) {
                logger.println("=== TRANSACTION END ===");
                logger.println();  // Blank line for readability
                logger.close();
                System.out.println("Transaction logged.");
            }
        }
    }

    public static void main(String[] args) {
        // Test with different amounts
        logTransaction("Gru", 5000);    // Should succeed
        logTransaction("Vector", 15000); // Should fail (too large)
        logTransaction("Scarlet", 7500); // Should succeed

        System.out.println("\nCheck transactions.log to see all attempts!");
    }
}

Writing TransactionLogger.java


In [8]:
!javac TransactionLogger.java
!java TransactionLogger

Transaction successful!
Transaction logged.
Transaction failed: Amount too large for standard transfer
Transaction logged.
Transaction successful!
Transaction logged.

Check transactions.log to see all attempts!



Run this program multiple times and check the log file. Notice that every transaction is properly logged with a complete start and end marker, whether it succeeds or fails. That's the power of `finally` - guaranteed cleanup!

## The Golden Rule

If you open it, close it. If you start it, finish it. If you lock it, unlock it. The `finally` block ensures these things happen no matter what goes wrong. It's like having a responsible minion who always turns off the lights and locks the door, even when the villain's plan goes completely wrong!